<a href="https://colab.research.google.com/github/FujiminoDeliaMary/ArtfullCode/blob/main/FouilleDonnee/3-ANN/TP3_ANN_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation des Librairies

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

## 1. Préparation des données

1.1 Importer les données Train et Test

In [7]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/My Drive/Data_Titanic/train_clean.csv')
test = pd.read_csv('/content/drive/My Drive/Data_Titanic/test_clean.csv')

train.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,1,0,3,1,1,0,0,0,1
1,2,1,1,0,2,3,1,0,3
2,3,1,3,0,1,1,0,1,2
3,4,1,1,0,2,3,0,0,3
4,5,0,3,1,2,1,0,1,1


In [8]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,892,3,1,2,0,2,1,1
1,893,3,0,2,0,0,0,3
2,894,2,1,3,1,2,1,1
3,895,3,1,1,1,0,1,1
4,896,3,0,1,1,0,0,3


1.2 Depuis le Dataframe train, charger les features d'apprentissage dans un array numpy X_alltrain, et les labels (données à prévoir) dans un array numpy y_alltrain

In [9]:
X_alltrain = train.drop(columns=["Survived"]).to_numpy()
y_alltrain = train["Survived"].to_numpy()

1.3 Séparer les features et les labels en deux parties (train et dev), en attribuant 10% des exemples aux données de dev. afficher les nombres de lignes et de colonnes pour les 4 arrays.

In [10]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=1)

print("X_train :\n", X_train.shape)
print("X_dev :\n", X_dev.shape)
print("Y_train :\n", Y_train.shape)
print("Y_dev :\n", Y_dev.shape)

X_train :
 (801, 8)
X_dev :
 (90, 8)
Y_train :
 (801,)
Y_dev :
 (90,)


1.4 Afficher les 10 premières lignes de features et les 10 premiers labels.

In [11]:
print(X_train[:10])

[[663   1   1   2   2   0   1   1]
 [585   3   1   1   1   1   1   1]
 [704   3   1   1   0   2   1   1]
 [ 48   3   0   1   0   2   1   2]
 [869   3   1   2   1   0   1   1]
 [453   1   1   1   2   1   1   1]
 [ 95   3   1   3   0   0   1   1]
 [ 20   3   0   2   0   1   1   3]
 [878   3   1   1   0   0   1   1]
 [879   3   1   1   0   0   1   1]]


In [ ]:
print(Y_train[:10])

[0 0 0 1 0 0 0 1 0 0]


## 2. Définition du réseau de neurone

2.1 Définir et instancier une classe Titanic Model avec les caractéristiques suivantes :
- deux  couches cachées de 50 neurones.
- deux classes en sortie : Survivant ou non
- Des fonctions d'activation RELU
- un dropout paramétrable pour les 2 couches cachées

In [13]:
class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
      super(TitanicModel, self).__init__()
      self.hidden1 = nn.Linear(input_size, 50)
      self.dropout1 = nn.Dropout(dropout_rate)
      self.hidden2 = nn.Linear(50, 50)
      self.dropout2 = nn.Dropout(dropout_rate)
      self.output = nn.Linear(50, 2)

    def forward(self, x):
      x = F.relu(self.hidden1(x))
      x = self.dropout1(x)
      x = F.relu(self.hidden2(x))
      x = self.dropout2(x)
      x = self.output(x)
      return x


input_size = X_train.shape[1]
dropout_rate = 0.5

model = TitanicModel(input_size, dropout_rate)

2.2 : Définir des paramètres de nombre d'epochs (50) et de learning_rate (0.01)

In [14]:
epochs = 50
lr = 0.01

2.3 définir la taille du minibatch à 50. En déduire le nombre de boucle pour chaque epoch.

In [15]:
batch_size = 50

2.4 Définir un fonction de coût de type CrossEntropy

In [16]:
cost_function = nn.CrossEntropyLoss()

2.5. Définir un optimizer de type Adam, sans oublier le learning rate

In [17]:
optimizer = optim.Adam(model.parameters(), lr=lr)

## 3. Apprentissage

3.1 exécuter l'apprentissage du modèle.
- Créer une boucle sur les epochs, qui contient elel-même une boucle sur les minibatchs.
- A chaque nouvelle itération sur les epochs, mélanger les données avec la méthode shuffle.
- Tous les 5 epochs afficher la valeur de la fonction de cout

In [24]:
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold

In [19]:


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = cost_function(outputs, Y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs}, Perte moyenne: {avg_loss:.4f}")

Epoch 5/50, Perte moyenne: 0.6859
Epoch 10/50, Perte moyenne: 0.6607
Epoch 15/50, Perte moyenne: 0.6594
Epoch 20/50, Perte moyenne: 0.6523
Epoch 25/50, Perte moyenne: 0.6517
Epoch 30/50, Perte moyenne: 0.6539
Epoch 35/50, Perte moyenne: 0.6801
Epoch 40/50, Perte moyenne: 0.6809
Epoch 45/50, Perte moyenne: 0.6548
Epoch 50/50, Perte moyenne: 0.6460


3.2 Calculer la précision de la prévision sur les données dev

In [21]:
X_dev_tensor = torch.tensor(X_dev, dtype=torch.float32)
Y_dev_tensor = torch.tensor(Y_dev, dtype=torch.long)

model.eval()
with torch.no_grad():
  logits = model(X_dev_tensor)
  predictions = torch.argmax(logits, dim=1)

accuracy = accuracy_score(Y_dev, predictions.numpy())
print(f"Précision : {accuracy:.4f}")

Précision : 0.6000


3.3 Calculer prévisions sur les données de tests

In [22]:
X_test_tensor = torch.tensor(test.values, dtype=torch.float32)
model.eval()
with torch.no_grad():
  logits_test = model(X_test_tensor)
  predictions_test = torch.argmax(logits_test, dim=1)
predictions_test = predictions_test.numpy()

3.4 Générer le fichier résultat et l'envoyer sur kaggle
Quel est votre score et votre classement ?

In [23]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predictions_test })

submission_file = "submission.csv"
submission.to_csv(submission_file, index=False)

In [29]:
#installation de kaggle
!pip install kaggle

In [30]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!ls -l ~/.kaggle

!kaggle competitions submit -c titanic -f submission.csv -m "première soumission"

!kaggle competitions submissions -c titanic
!kaggle competitions leaderboard titanic --download --csv

cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
total 0
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-

3.5 (Optionnel) Exécuter une Cross Validationd ans une boucle pour trouver les meilleures valeurs de learning rate, de keep_prob et de nombre d'epochs.

Cross_Validation avec skkearn : https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [25]:
learning_rates = [0.001, 0.01, 0.1]
keep_probs = [0.3, 0.5, 0.7]
epochs_options = [20, 50, 100]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_score = 0
best_params = {}